In [12]:
import pandas as pd
import numpy as np

In [13]:
price_df = pd.read_csv("ind_stock_return_final_0924.csv")
print(price_df.shape)
price_df.head()

(1955057, 14)


,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,iid,datadate,tic,prccd,prcod,ccmbegdt
0,1045,P,1,LC,21020,20010,19620131,20120104,1,20100104,AAMRQ,7.72,7.73,20090201
1,1045,P,1,LC,21020,20010,19620131,20120104,1,20100105,AAMRQ,8.36,7.68,20090201
2,1045,P,1,LC,21020,20010,19620131,20120104,1,20100106,AAMRQ,8.11,8.11,20090201
3,1045,P,1,LC,21020,20010,19620131,20120104,1,20100107,AAMRQ,8.25,8.05,20090201
4,1045,P,1,LC,21020,20010,19620131,20120104,1,20100108,AAMRQ,8.11,8.34,20090201


In [14]:
price_df = price_df.drop(['GVKEY','LINKPRIM','LIID',
                          'LINKTYPE','LPERMNO','LPERMCO','LINKDT','LINKENDDT',
                          'iid','ccmbegdt'], axis = 1)
print(price_df.shape)
price_df.head()

(1955057, 4)


,datadate,tic,prccd,prcod
0,20100104,AAMRQ,7.72,7.73
1,20100105,AAMRQ,8.36,7.68
2,20100106,AAMRQ,8.11,8.11
3,20100107,AAMRQ,8.25,8.05
4,20100108,AAMRQ,8.11,8.34


In [15]:
print("Number of unique Ticker Symbols:", len(price_df.tic.unique()))

Number of unique Ticker Symbols: 705


In [16]:
price_df = price_df.sort_values(by=['tic', 'datadate'], 
                                ascending=True).reset_index(drop=True).set_index(['tic','datadate'])
print(price_df.shape)
price_df.head()

(1955057, 2)


prccd  prcod
tic datadate              
A   20100104  31.30  31.39
    20100105  30.96  31.21
    20100106  30.85  30.85
    20100107  30.81  30.78
    20100108  30.80  30.64

In [17]:
lead_var = price_df.groupby(level="tic")['prccd'].shift(-2)
price_df['prccd_(t+2)'] = lead_var

print(price_df.shape)
price_df.head()

(1955057, 3)


prccd  prcod  prccd_(t+2)
tic datadate                           
A   20100104  31.30  31.39        30.85
    20100105  30.96  31.21        30.81
    20100106  30.85  30.85        30.80
    20100107  30.81  30.78        30.82
    20100108  30.80  30.64        30.45

In [18]:
price_df['3_day_buy_and_hold_return'] = (price_df['prccd_(t+2)'] - price_df['prcod'])/price_df['prcod']
price_df['3_day_buy_and_hold_log_return'] = np.log(price_df['prccd_(t+2)']/price_df['prcod'])
print(price_df.shape)
price_df.head()

(1955057, 5)


prccd  prcod  prccd_(t+2)  3_day_buy_and_hold_return  \
tic datadate                                                         
A   20100104  31.30  31.39        30.85                  -0.017203   
    20100105  30.96  31.21        30.81                  -0.012816   
    20100106  30.85  30.85        30.80                  -0.001621   
    20100107  30.81  30.78        30.82                   0.001300   
    20100108  30.80  30.64        30.45                  -0.006201   

              3_day_buy_and_hold_log_return  
tic datadate                                 
A   20100104                      -0.017353  
    20100105                      -0.012899  
    20100106                      -0.001622  
    20100107                       0.001299  
    20100108                      -0.006220

In [19]:
index_return_df = pd.read_csv("index_return_SnP.csv")
print(index_return_df.shape)
index_return_df.head()

(2517, 2)


,datadate,vwretd
0,20120103,0.015379
1,20120104,0.000388
2,20120105,0.002898
3,20120106,-0.002191
4,20120109,0.002330


In [20]:
index_return_df['3_day_buy_and_hold_index_log_return'] = index_return_df['vwretd'].shift(0) \
                            + index_return_df['vwretd'].shift(-1) + index_return_df['vwretd'].shift(-2)

index_return_df['3_day_buy_and_hold_index_return'] = (1 + index_return_df['vwretd'].shift(0)) \
                            * (1 + index_return_df['vwretd'].shift(-1)) \
                            * (1 + index_return_df['vwretd'].shift(-2)) - 1

print(index_return_df.shape)
index_return_df.head()

(2517, 4)


,datadate,vwretd,3_day_buy_and_hold_index_log_return,3_day_buy_and_hold_index_return
0,20120103,0.015379,0.018665,0.018717
1,20120104,0.000388,0.001095,0.001089
2,20120105,0.002898,0.003037,0.003032
3,20120106,-0.002191,0.009073,0.009069
4,20120109,0.002330,0.011677,0.011702


In [21]:
final_return_df = pd.merge(price_df.reset_index(), index_return_df, how = 'left', on='datadate')
print(final_return_df.shape)
final_return_df.head()

(1955057, 10)


,tic,datadate,prccd,prcod,prccd_(t+2),3_day_buy_and_hold_return,3_day_buy_and_hold_log_return,vwretd,3_day_buy_and_hold_index_log_return,3_day_buy_and_hold_index_return
0,A,20100104,31.30,31.39,30.85,-0.017203,-0.017353,NaN,NaN,NaN
1,A,20100105,30.96,31.21,30.81,-0.012816,-0.012899,NaN,NaN,NaN
2,A,20100106,30.85,30.85,30.80,-0.001621,-0.001622,NaN,NaN,NaN
3,A,20100107,30.81,30.78,30.82,0.001300,0.001299,NaN,NaN,NaN
4,A,20100108,30.80,30.64,30.45,-0.006201,-0.006220,NaN,NaN,NaN


In [22]:
final_return_df['3_day_buy_and_hold_excess_return'] = \
    final_return_df['3_day_buy_and_hold_return'] - final_return_df['3_day_buy_and_hold_index_return']

final_return_df['3_day_buy_and_hold_excess_log_return'] = \
    final_return_df['3_day_buy_and_hold_log_return'] - final_return_df['3_day_buy_and_hold_index_log_return']

print(final_return_df.shape)
final_return_df.head()

(1955057, 12)


,tic,datadate,prccd,prcod,prccd_(t+2),3_day_buy_and_hold_return,3_day_buy_and_hold_log_return,vwretd,3_day_buy_and_hold_index_log_return,3_day_buy_and_hold_index_return,3_day_buy_and_hold_excess_return,3_day_buy_and_hold_excess_log_return
0,A,20100104,31.30,31.39,30.85,-0.017203,-0.017353,NaN,NaN,NaN,NaN,NaN
1,A,20100105,30.96,31.21,30.81,-0.012816,-0.012899,NaN,NaN,NaN,NaN,NaN
2,A,20100106,30.85,30.85,30.80,-0.001621,-0.001622,NaN,NaN,NaN,NaN,NaN
3,A,20100107,30.81,30.78,30.82,0.001300,0.001299,NaN,NaN,NaN,NaN,NaN
4,A,20100108,30.80,30.64,30.45,-0.006201,-0.006220,NaN,NaN,NaN,NaN,NaN


In [23]:
final_return_df.to_csv("cleaned_stock_excess_return_final_0924.csv", index=False)

In [24]:
final_return_df['datadate'] = pd.to_datetime(final_return_df['datadate'], format= '%Y%m%d')

print(final_return_df.shape)
final_return_df.head()

(1955057, 12)


,tic,datadate,prccd,prcod,prccd_(t+2),3_day_buy_and_hold_return,3_day_buy_and_hold_log_return,vwretd,3_day_buy_and_hold_index_log_return,3_day_buy_and_hold_index_return,3_day_buy_and_hold_excess_return,3_day_buy_and_hold_excess_log_return
0,A,2010-01-04,31.30,31.39,30.85,-0.017203,-0.017353,NaN,NaN,NaN,NaN,NaN
1,A,2010-01-05,30.96,31.21,30.81,-0.012816,-0.012899,NaN,NaN,NaN,NaN,NaN
2,A,2010-01-06,30.85,30.85,30.80,-0.001621,-0.001622,NaN,NaN,NaN,NaN,NaN
3,A,2010-01-07,30.81,30.78,30.82,0.001300,0.001299,NaN,NaN,NaN,NaN,NaN
4,A,2010-01-08,30.80,30.64,30.45,-0.006201,-0.006220,NaN,NaN,NaN,NaN,NaN


In [28]:
report_10qk_data = pd.read_csv("merged.csv")
report_10qk_data['date'] = pd.to_datetime(report_10qk_data['date'])

print(report_10qk_data.shape)
report_10qk_data.head()

(12546, 4)


,ticker,type,date,file
0,A,10-Q,2021-09-01,united states securities exchange commission w...
1,A,10-Q,2021-06-01,united states securities exchange commission w...
2,A,10-Q,2021-03-02,united states securities exchange commission w...
3,A,10-Q,2020-09-01,united states securities exchange commission w...
4,A,10-Q,2020-06-01,united states securities exchange commission w...


In [29]:
final_merged_data = pd.merge(report_10qk_data, final_return_df, how='left', left_on=['ticker', 'date'],
                            right_on = ['tic', 'datadate'])
print(final_merged_data.shape)
final_merged_data.head()

(12546, 16)


,ticker,type,date,file,tic,datadate,prccd,prcod,prccd_(t+2),3_day_buy_and_hold_return,3_day_buy_and_hold_log_return,vwretd,3_day_buy_and_hold_index_log_return,3_day_buy_and_hold_index_return,3_day_buy_and_hold_excess_return,3_day_buy_and_hold_excess_log_return
0,A,10-Q,2021-09-01,united states securities exchange commission w...,A,2021-09-01,175.45,175.04,179.28,0.024223,0.023934,0.000448,0.003120,0.003121,0.021102,0.020814
1,A,10-Q,2021-06-01,united states securities exchange commission w...,A,2021-06-01,137.20,139.17,136.37,-0.020119,-0.020324,-0.000367,-0.002511,-0.002516,-0.017604,-0.017813
2,A,10-Q,2021-03-02,united states securities exchange commission w...,A,2021-03-02,123.64,124.57,114.99,-0.076905,-0.080023,-0.008142,-0.034538,-0.034150,-0.042754,-0.045485
3,A,10-Q,2020-09-01,united states securities exchange commission w...,A,2020-09-01,101.12,100.92,99.64,-0.012683,-0.012764,0.007952,-0.011652,-0.012349,-0.000335,-0.001112
4,A,10-Q,2020-06-01,united states securities exchange commission w...,A,2020-06-01,89.91,88.04,90.49,0.027828,0.027448,0.003979,0.025656,0.025853,0.001976,0.001792


In [30]:
final_merged_data.to_csv("final_merged_data_for_project_1_v2.csv", index=False)

In [34]:
len(final_merged_data[pd.isnull(final_merged_data['tic'])==True]['ticker'].unique())

79